<h3> Intro API </h3>

- A set of commands, protocols, and objects that programmers can use to create software or interact with an external system
- Through API our program can request data from an external system and if we follow the rules it has prescribed ES will respond
- We can't just get into the kitchen of a restaurant to fiddle around, jsut like restaurant has a menu, similarly API has a menu
    - It will tell you what all you can get and what all you can do through the API documentation
- API endpoint is the web location at which you make the request for your data
- REST (Representational State Transfer) API or RESTful API delivers the request in a way that client can understand and use in a standardised format
- Most freqeunt HTTP methods in a restful API
    - Get - Retrieves a resource
    - POST - Creates a resource
    - PUT - updates an existing resource
    - DELETE - removes a resource
- Using the ISS spacestation API endpoint
    - Available at http://open-notify.org/Open-Notify-API/ISS-Location-Now/
    - When you use the API endpoint http://api.open-notify.org/iss-now.json in browser you will get data in json format
    - {"message": "success", "timestamp": 1741660329, "iss_position": {"longitude": "175.3226", "latitude": "-44.2287"}}
- Need to use 'requests' library
- **Types of response codes**
    - 1XX - Hold On
    - 2XX - Here you go , response successful
    - 3XX - Go away you dont have permission
    - 4XX - Object not found/present
    - 5XX - Server is down, website is down

 

In [9]:
import requests

response =  requests.get(url = "http://api.open-notify.org/iss-now.json") ## adding the API endpoint in the get method for requests module and storing the output in response variable

##instread of having to call for exceptiosn for all type of HTTP , we can use taise_for_status() method
# if response.status_code == 404:
#     raise Exception("That resource does not exist")
# elif response.status_code == 401:
#     raise Exception("You are not authorised to access this data")

response.raise_for_status() ## this will lead to an error

In [5]:
print(response) ## We get 200 which means call was succesful

<Response [200]>


In [14]:
## to actually get the data , we have to invoke the json method which parses response call into readable json data

data = response.json()
longitude = data["iss_position"]["longitude"]
latitude = data["iss_position"]["latitude"]
iss_position = (longitude, latitude)
print(iss_position)

('-115.2728', '28.5541')


<h3> Kanye Quote Machine </h3>

In [ ]:
from tkinter import *
import requests

def get_quote():
    response = requests.get(url = "https://api.kanye.rest")
    quote = response.json()["quote"]
    canvas.itemconfig(quote_text, text=quote)
                            



window = Tk()
window.title("Kanye Says...")
window.minsize(width = 600, height = 600)
window.config(padx = 200, pady = 50)


text_label = Label(text = "")
text_label.grid(row = 0, column = 1)

box = PhotoImage(file = "Day33Kanye/background.png")
canvas = Canvas(height = box.height(), width = box.width())
canvas.create_image(box.width()//2, box.height()//2, image = box)
quote_text =  canvas.create_text(box.width()//2, box.height()//2,  width=box.width() - 100, text = "Press button to get quote", font = ("Arial", 24, "bold"))
canvas.grid(row  = 1, column = 1)

kanye = PhotoImage(file = "Day33Kanye/kanye.png")
kanye_button = Button(image = kanye, command = get_quote )
kanye_button.grid(row =2  , column = 1)



window.mainloop()

In [1]:
import requests
response = requests.get(url = "https://api.kanye.rest")
response.json()["quote"]

'Culture is the most powerful force in humanity under God'

<h3> Understanding API params </h3>

- For e.g if we want to check for sunrise time or sunset time then it will be dependent on the location
- Then lat and long will be the mandatory params, and others can be optional params


In [7]:
import requests
from datetime import datetime

MY_LAT = 26.912434
MY_LNG = 75.787270

params = {"lat": MY_LAT,
         "lng": MY_LNG,
         "formatted": 0}

response =  requests.get(url = "https://api.sunrise-sunset.org/json", params = params) ## need to specify for params
response.raise_for_status()
data = response.json()
sunrise = data["results"]["sunrise"]
sunset = data["results"]["sunset"]
print(sunrise.split("T")[1].split(":")[0]) ##will return a list of strings separated by T ## to get actual hour in 24 ohur clock

time_now = datetime.now()

01


<h3> ISS Overhead Notifier Project </h3>

- If the ISS is close to my current position
- And it is currently dark
- Then send me an email to look up
- Bonus, run the code every 60 seconds 

In [19]:
import smtplib 
import random
import datetime as dt

def email_lookup():
    my_email = "shubham07iitr@gmail.com"
    password = "cifhsoqrbycnefnt" ## this is an app password which we have to 

    with  smtplib.SMTP("smtp.gmail.com") as connection: ## Creating a new objet to conect to our email providers SMTP email server , text in quotes is the olocation of our email providers SMTP server
        connection.starttls()
        ##tls = transfer layer securtity is  a way of securing our connection - encrypting our message and securing our connection
        connection.login(user = my_email, password = password)
        connection.sendmail(
            from_addr= my_email, 
            to_addrs= "shubham07iitr@gmail.com",
            msg = f"Subject:ISS Flying Up\n\nPlease look up, ISS flying"
        )


def check_email():
    response_iss = requests.get(url = "http://api.open-notify.org/iss-now.json")
    data = response_iss.json()
    long_check = float(data["iss_position"]["longitude"])  >= 26.912434 - 5 and float(data["iss_position"]["longitude"]) <= 26.912434 + 5
    lat_check = float(data["iss_position"]["latitude"])  >= 75.787270 - 5 and float(data["iss_position"]["latitude"]) <=  75.787270 + 5
    time_check = datetime.now().hour  in (19, 20, 21, 22, 23, 1, 2, 3, 4, 5,6)
    if long_check and lat_check  and time_check:
        print("Wooho! Go and check outside")
        email_lookup()
    else:
        print("Sorry, no ISS for you")
    

check_email()
    

Sorry, no ISS for you


In [10]:
type(datetime.now().hour)

int

In [13]:
time_check = datetime.now().hour  in (19, 20, 21, 22, 23, 1, 2, 3, 4, 5,6)
time_check

False

In [18]:
response_iss = requests.get(url = "http://api.open-notify.org/iss-now.json")
data = response.json()
data
# long_check = float(data["iss_position"]["longitude"])  >= 26.912434 - 5 and float(data["iss_position"]["longitude"]) <= 26.912434 + 5
# long_check

{'results': {'sunrise': '2025-03-11T01:09:09+00:00',
  'sunset': '2025-03-11T13:04:29+00:00',
  'solar_noon': '2025-03-11T07:06:49+00:00',
  'day_length': 42920,
  'civil_twilight_begin': '2025-03-11T00:47:10+00:00',
  'civil_twilight_end': '2025-03-11T13:26:29+00:00',
  'nautical_twilight_begin': '2025-03-11T00:20:15+00:00',
  'nautical_twilight_end': '2025-03-11T13:53:24+00:00',
  'astronomical_twilight_begin': '2025-03-10T23:53:16+00:00',
  'astronomical_twilight_end': '2025-03-11T14:20:22+00:00'},
 'status': 'OK',
 'tzid': 'UTC'}